In [1]:
# Python script that mirrors the steps in the fold mirror example (Example 10.3) using aluminum at 45° AOI

import numpy as np
from scipy.linalg import svd
import pandas as pd

# STEP 1: Define Fresnel coefficients for aluminum at 800 nm and 45° AOI
# These complex values represent amplitude reflectance for p and s polarizations
r_s = 0.91 * np.exp(1j * np.deg2rad(5))    # s-polarization
r_p = 0.87 * np.exp(1j * np.deg2rad(25))   # p-polarization

# STEP 2: Build Jones matrix in s–p basis
J = np.array([[r_p, 0],
              [0, r_s]])

# STEP 3: Compute cumulative Jones matrix for two mirrors (mirror 2 * mirror 1)
J_total = J @ J

# STEP 4: Calculate retardance using Q.T @ P method (Equation 10.38)
def compute_retardance_sp_only(J):
    Q = np.diag([1, -1])  # Geometric transformation matrix for reflection
    QTP = Q.T @ J
    evals = np.linalg.eigvals(QTP)
    phases = np.angle(evals)
    delta = np.abs(phases[0] - phases[1])
    delta = np.mod(delta + np.pi, 2 * np.pi) - np.pi
    return np.rad2deg(delta), evals

ret1_deg, _ = compute_retardance_sp_only(J)
ret2_deg, _ = compute_retardance_sp_only(J)
ret_total_deg, _ = compute_retardance_sp_only(J_total)

# STEP 5: Convert Jones to Mueller matrix using Pauli matrices (Equation 10.37 style)
def jones_to_mueller(J):
    sigma = [
        np.array([[1, 0], [0, 1]]),   # Identity
        np.array([[1, 0], [0, -1]]),  # Z
        np.array([[0, 1], [1, 0]]),   # X
        np.array([[0, -1j], [1j, 0]]) # Y
    ]
    M = np.zeros((4, 4))
    for i in range(4):
        for j in range(4):
            M[i, j] = 0.5 * np.real(np.trace(sigma[i] @ J @ sigma[j] @ J.conj().T))
    return M

M1 = jones_to_mueller(J)
M2 = jones_to_mueller(J)
M_total = jones_to_mueller(J_total)

# STEP 6: Extract diattenuation, polarizance, and rotation (like in bottom of page 2 / top of page 3)
def extract_properties(M):
    D_vec = M[0, 1:4]
    D_mag = np.linalg.norm(D_vec)
    P_vec = M[1:4, 0]
    P_mag = np.linalg.norm(P_vec)
    M33 = M[1:4, 1:4]
    U, S, Vh = svd(M33)
    R_matrix = U @ Vh
    rot_angle = np.arccos((np.trace(R_matrix) - 1) / 2)
    rot_angle = np.degrees(rot_angle)
    return D_mag, P_mag, rot_angle, S

# Collect properties
rows = []
for name, M, ret in zip(["Mirror 1", "Mirror 2", "Combined"],
                        [M1, M2, M_total],
                        [ret1_deg, ret2_deg, ret_total_deg]):
    D, P, rot, eigs = extract_properties(M)
    rows.append({
        "Element": name,
        "Diattenuation": D,
        "Polarizance": P,
        "Rotation (deg)": rot,
        "Retardance (deg)": ret,
        "Submatrix Eigenvalues": eigs
    })

df_summary = pd.DataFrame(rows).set_index("Element")

# Save Mueller matrices
M_stack = np.vstack([M1, M2, M_total])
df_mueller = pd.DataFrame(M_stack,
                          index=[f"M1_{i}" for i in range(4)] +
                                [f"M2_{i}" for i in range(4)] +
                                [f"Mtotal_{i}" for i in range(4)],
                          columns=[f"Col_{j}" for j in range(4)])
print(df_mueller)
csv_path = "periscope_mirrors_mueller_analysis_output.csv"
df_mueller.to_csv(csv_path)

df_summary, csv_path


             Col_0     Col_1     Col_2     Col_3
M1_0      0.792500 -0.035600  0.000000  0.000000
M1_1     -0.035600  0.792500  0.000000  0.000000
M1_2      0.000000  0.000000  0.743955  0.270777
M1_3      0.000000  0.000000 -0.270777  0.743955
M2_0      0.792500 -0.035600  0.000000  0.000000
M2_1     -0.035600  0.792500  0.000000  0.000000
M2_2      0.000000  0.000000  0.743955  0.270777
M2_3      0.000000  0.000000 -0.270777  0.743955
Mtotal_0  0.629324 -0.056426  0.000000  0.000000
Mtotal_1 -0.056426  0.629324  0.000000  0.000000
Mtotal_2  0.000000  0.000000  0.480148  0.402892
Mtotal_3  0.000000  0.000000 -0.402892  0.480148


(          Diattenuation  Polarizance  Rotation (deg)  Retardance (deg)  \
 Element                                                                  
 Mirror 1       0.035600     0.035600            20.0            -160.0   
 Mirror 2       0.035600     0.035600            20.0            -160.0   
 Combined       0.056426     0.056426            40.0            -140.0   
 
                                  Submatrix Eigenvalues  
 Element                                                 
 Mirror 1          [0.7925, 0.7917000000000001, 0.7917]  
 Mirror 2          [0.7925, 0.7917000000000001, 0.7917]  
 Combined  [0.6293236099999999, 0.62678889, 0.62678889]  ,
 'periscope_mirrors_mueller_analysis_output.csv')